To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
import torch
!!pip install -U datasets
from datasets import  concatenate_datasets , load_dataset
import huggingface_hub
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import re

from datasets import  concatenate_datasets , load_dataset
import huggingface_hub

!pip install huggingface_hub
from google.colab import userdata
token = userdata.get('comp')
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token(f'{token}')"

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Installation

In [ ]:
!gdown 18_pZMpxI2nwBn6kfz60OSFlXDWw5Pz29 --folder

Retrieving folder contents
Processing file 16rK23V-PYk60kxximk8kIahHpSz1XBWQ Task1_MCQs_Dev.csv
Processing file 1qBzjTVx0g6kav3sXVv-4CJq43rjuCIKf Task1_MCQs_Test.csv
Processing file 1qi994X3HnPOmoPkcI28PmQGfY1Zyexjo Task1_MCQs_Train_batch2.csv
Processing file 1eyBW9XnZaRGPMCW90gfxvZospaQa9rNG Task1_MCQs_Train.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=16rK23V-PYk60kxximk8kIahHpSz1XBWQ
To: /content/Qias/Task1_MCQs_Dev.csv
100% 396k/396k [00:00<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qBzjTVx0g6kav3sXVv-4CJq43rjuCIKf
To: /content/Qias/Task1_MCQs_Test.csv
100% 1.48M/1.48M [00:00<00:00, 138MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qi994X3HnPOmoPkcI28PmQGfY1Zyexjo
To: /content/Qias/Task1_MCQs_Train_batch2.csv
100% 13.3M/13.3M [00:00<00:00, 91.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eyBW9XnZaRGPMCW90gfxvZospaQa9

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'EmanElrefai/Islamic_heritage_epoch20',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token =''
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-26 00:57:45 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-26 00:57:45 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.53.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.7.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 512,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 512,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [ ]:
reasoning_start = "<start_reasoning>" # Acts as <think>
reasoning_end   = "<end_reasoning>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين {reasoning_start} و {reasoning_end}.
ثم، قدم الحل بين {solution_start}{solution_end}"""
system_prompt

'أنت أمام مسألة معينة.\nفكّر في المسألة وقدم شرح خطوات تفكيرك.\nضعها بين <start_reasoning> و <end_reasoning>.\nثم، قدم الحل بين <SOLUTION></SOLUTION>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [ ]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [ ]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"أنت أمام مسألة معينة.\nفكّر في المسألة وقدم شرح خطوات تفكيرك.\nضعها بين <start_reasoning> و <end_reasoning>.\nثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>What is 1+1?<start_reasoning>I think it's 2.<end_reasoning><SOLUTION>2</SOLUTION><|im_end|>What is 2+2?<start_reasoning>"

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>

### Pre fine-tuning for formatting

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict
import os

def load_and_split_instruct_dataset(dataset_path, test_size=0.1, seed=42):
    """Loads the instruct dataset and splits it into train and dev sets."""
    dataset = load_dataset("csv", data_files={"train": dataset_path})
    split_dataset = dataset["train"].train_test_split(test_size=test_size, seed=seed)
    return split_dataset["train"], split_dataset["test"]

def load_dataset_from_path(dataset_path, split="train"):
    """Loads a dataset from a given path."""
    return load_dataset("csv", data_files={split: dataset_path}, split=split)

def concatenate_datasets_list(datasets_list):
    """Concatenates a list of datasets."""
    return concatenate_datasets(datasets_list)

root_path = "./Qias"

dataset_instruct_path = os.path.join(root_path, "Task1_MCQs_Train_batch2.csv")
train_dataset_mcq_path = os.path.join(root_path, "Task1_MCQs_Train.csv")
dev_dataset_path = os.path.join(root_path, "Task1_MCQs_Dev.csv")
test_dataset_path = os.path.join(root_path, "Task1_MCQs_Test.csv")

# Load and split the instruct dataset
train_dataset_instruct, dev_dataset_instruct = load_and_split_instruct_dataset(dataset_instruct_path, test_size=0.1)

# Load the MCQ only training dataset
train_dataset_mcq_only = load_dataset_from_path(train_dataset_mcq_path, split="train")

# Concatenate instruct and MCQ only datasets for the full training set
train_dataset_mcq = concatenate_datasets_list([train_dataset_instruct, train_dataset_mcq_only])

# Load dev and test datasets
dev_dataset_mcq = load_dataset_from_path(dev_dataset_path, split="dev")
test_dataset_mcq = load_dataset_from_path(test_dataset_path, split="test")


print("Train Dataset Instruct:")
print(train_dataset_instruct)
print("\nDev Dataset Instruct:")
print(dev_dataset_instruct)

print("\nTrain Dataset MCQ:")
print(train_dataset_mcq, "\n")
print("Dev Dataset MCQ:")
print(dev_dataset_mcq, "\n")
print("Test Dataset MCQ:")
print(test_dataset_mcq, "\n")

Train Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level'],
    num_rows: 9498
})

Dev Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level'],
    num_rows: 1056
})

Train Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level'],
    num_rows: 18944
}) 

Dev Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level'],
    num_rows: 1000
}) 

Test Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'level'],
    num_rows: 1000
}) 



We have to format the dataset to follow our GRPO style formatting:

In [ ]:
def format_dataset(x):
    expected_answer = x["label"]
    problem = x["question"]

    label2col = {
        "A" : "option1",
        "B" : "option2",
        "C" : "option3",
        "D" : "option4",
        "E" : "option5",
        "F" : "option6",
    }

    # Remove generated <think> and </think>
    thoughts = x[label2col[expected_answer]]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    formatted_prompt_string = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end

    # Return both the original messages structure and the formatted string
    return {
        "Messages" : [
            {"role" : "system",    "content" : system_prompt},
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : formatted_prompt_string},
        ]
    }

train_dataset_instruct = train_dataset_instruct.map(format_dataset)
dev_dataset_instruct = dev_dataset_instruct.map(format_dataset)

train_dataset_mcq = train_dataset_mcq.map(format_dataset)
dev_dataset_mcq = dev_dataset_mcq.map(format_dataset)

Map:   0%|          | 0/9498 [00:00<?, ? examples/s]

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/18944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Check to see if it worked:

In [ ]:
print(tokenizer.apply_chat_template(train_dataset_instruct["Messages"][0], tokenize = False))

أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين <start_reasoning> و <end_reasoning>.
ثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>مات وترك: أخت لأم و أخ شقيق (2) و ابن عم الأب (3) و ابن ابن عم لأب (4) و أم الأب و أم أب الأب كم النصيب الأصلي لـ أخت لأم من التركة، وما الدليل على ذلك؟<start_reasoning>نصيبه هو السدس، والدليل: فرض الواحد - المنفرد - من الأخوة لأم هو السدس سواء أكان ذكرا أو أنثى ، وشرط ميراثهم هو عدم وجود الفرع الوارث المذكر - مثل الإبن وابن الإبن - أو المؤنث - مثل البنت وبنت الإبن - وعدم وجود الأصل الوارث المذكر - مثل الأب و أب الأب<end_reasoning><SOLUTION>A</SOLUTION><|im_end|>


Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [ ]:
from datasets import Dataset
import pandas as pd

def process_dataset_length(dataset, tokenizer, max_seq_length):
    # Calculate the length of the formatted messages
    dataset = dataset.map(lambda x: {"N": len(tokenizer.apply_chat_template(x["Messages"], tokenize=False))})

    # Convert to pandas DataFrame for filtering
    dataset_df = dataset.to_pandas()

    # Filter based on length
    dataset_df = dataset_df.loc[dataset_df["N"] <= max_seq_length/2].copy()

    # Convert back to Hugging Face Dataset
    dataset = Dataset.from_pandas(dataset_df)

    return dataset

train_dataset_instruct = process_dataset_length(train_dataset_instruct, tokenizer, max_seq_length)
dev_dataset_instruct = process_dataset_length(dev_dataset_instruct, tokenizer, max_seq_length)

train_dataset_mcq = process_dataset_length(train_dataset_mcq, tokenizer, max_seq_length)
dev_dataset_mcq = process_dataset_length(dev_dataset_mcq, tokenizer, max_seq_length)

print("Train Dataset Instruct:")
print(train_dataset_instruct)
print("\nDev Dataset Instruct:")
print(dev_dataset_instruct)

print("\nTrain Dataset MCQ:")
print(train_dataset_mcq, "\n")
print("Dev Dataset MCQ:")
print(dev_dataset_mcq, "\n")

Map:   0%|          | 0/9498 [00:00<?, ? examples/s]

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/18944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Train Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__'],
    num_rows: 9498
})

Dev Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__'],
    num_rows: 1056
})

Train Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__'],
    num_rows: 18944
}) 

Dev Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__'],
    num_rows: 1000
}) 



We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [ ]:
from datasets import Dataset
import pandas as pd

def add_formatted_text(dataset, tokenizer):
    # Ensure the dataset is a pandas DataFrame before applying the function
    if not isinstance(dataset, pd.DataFrame):
        dataset = dataset.to_pandas()

    dataset["prompt"] = dataset["Messages"].apply(lambda x: tokenizer.apply_chat_template(x, tokenize = False))
    return Dataset.from_pandas(dataset)

train_dataset_instruct = add_formatted_text(train_dataset_instruct, tokenizer)
dev_dataset_instruct = add_formatted_text(dev_dataset_instruct, tokenizer)

train_dataset_mcq = add_formatted_text(train_dataset_mcq, tokenizer)
dev_dataset_mcq = add_formatted_text(dev_dataset_mcq, tokenizer)

print("Train Dataset Instruct:")
print(train_dataset_instruct)
print("\nDev Dataset Instruct:")
print(dev_dataset_instruct)

print("\nTrain Dataset MCQ:")
print(train_dataset_mcq, "\n")
print("Dev Dataset MCQ:")
print(dev_dataset_mcq, "\n")

Train Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__', 'prompt'],
    num_rows: 9498
})

Dev Dataset Instruct:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__', 'prompt'],
    num_rows: 1056
})

Train Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__', 'prompt'],
    num_rows: 18944
}) 

Dev Dataset MCQ:
Dataset({
    features: ['id_question', 'question', 'option1', 'option2', 'option3', 'option4', 'option5', 'option6', 'label', 'level', 'Messages', 'N', '__index_level_0__', 'prompt'],
    num_rows: 1000
}) 



Let's check if the model has learnt to follow the custom format:

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

### Data Prep
<a name="Data"></a>

Let's look at the first row:

In [ ]:
train_dataset_instruct[0]["question"]

'مات وترك: أخت لأم و أخ شقيق (2) و ابن عم الأب (3) و ابن ابن عم لأب (4) و أم الأب و أم أب الأب كم النصيب الأصلي لـ أخت لأم من التركة، وما الدليل على ذلك؟'

In [ ]:
train_dataset_instruct[0]["label"]

'A'

We create a regex format to match the reasoning sections and answers:

In [ ]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_reasoning>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|im_end\|>)?[\s]{0,}$',
re.MULTILINE|re.DOTALL|re.UNICODE)

We verify it works:

In [ ]:
match_format.findall(
    f"Let me think!<{reasoning_end}>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [ ]:
match_format.findall(
    f"<{reasoning_start}>Let me think!<{reasoning_end}>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

['  2  ']

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for response in completions: # Assume completions is a list of strings
        score = 0
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

If it fails, we want to reward the model if it at least follows the format partially, by counting each symbol:

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for response in completions: # Assume completions is a list of strings
        score = 0
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [ ]:
def check_answer(prompts, completions, **kwargs): # Removed 'answer' as a positional argument
    # prompts and answer might still be lists of lists/other structures
    # based on how GRPOTrainer passes them. Let's keep them as is for now.
    # Assuming answer is a list of strings for each example in the batch.
    responses = completions # Assume completions is a list of strings

    # Access the ground truth answer from kwargs
    # Assuming it's passed under the key 'answer' or 'label'
    # Let's try 'label' first as that's the original column name
    true_answers = kwargs.get('label')
    if true_answers is None:
         # Fallback or raise an error if 'label' is not in kwargs
         # For now, let's assume it must be there and might be under 'answer'
         true_answers = kwargs.get('answer')
         if true_answers is None:
              # If still None, there's an issue with how trainer passes data
              print("Warning: Ground truth answers ('label' or 'answer' key) not found in kwargs.")
              return [-10.0] * len(completions) # Return a penalty if answers are missing

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, true_answers):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        elif guess.strip() in ["A", "B", "C", "D", "E", "F"]:
            score -= 3
        else:
            score -= 4.5 # Penalize
        scores.append(score)
    return scores

Also sometimes it might not be 1 number as the answer, but like a sentence for example "The solution is $20" -> we extract 20.

We also remove possible commas for example as in 123,456

In [ ]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


We now prepare our main function which will print out the generated responses and the true answer, along with another reward function which converts text to float via `float` and sees if it's the same.

In [ ]:
import numpy as np # Ensure numpy is imported if needed in this function

global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, **kwargs): # Removed 'answer' as a positional argument
    # prompts and answer might still be lists of lists/other structures
    # based on how GRPOTrainer passes them. Let's keep them as is for now.
    # Assuming answer is a list of strings for each example in the batch.
    # question = prompts[0][-1]["content"] # Still assuming prompts has the original structure
    responses = completions # Assume completions is a list of strings

    # Access the ground truth answer from kwargs
    # Assuming it's passed under the key 'answer' or 'label'
    # Let's try 'label' first as that's the original column name
    true_answers = kwargs.get('label')
    if true_answers is None:
         # Fallback or raise an error if 'label' is not in kwargs
         # For now, let's assume it must be there and might be under 'answer'
         true_answers = kwargs.get('answer')
         if true_answers is None:
              # If still None, there's an issue with how trainer passes data
              print("Warning: Ground truth answers ('label' or 'answer' key) not found in kwargs.")
              return [-10.0] * len(completions) # Return a penalty if answers are missing


    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    # Check if PRINTED_TIMES is multiple of PRINT_EVERY_STEPS and if prompts and responses are not empty
    # Also check if true_answers is not None and not empty before trying to access answer[0]
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0 and prompts and responses and extracted_responses and true_answers and len(true_answers) > 0:
        try:
            # Access question from prompts structure - assuming prompts[0] is the first prompt list
            # The structure of prompts might vary based on the trainer, so this might need adjustment
            # Let's try accessing the last message content as the question
            question_text = "Could not extract question text"
            if prompts and len(prompts) > 0 and isinstance(prompts[0], list) and len(prompts[0]) > 0 and isinstance(prompts[0][-1], dict) and "content" in prompts[0][-1]:
                 question_text = prompts[0][-1]["content"]
            elif prompts and len(prompts) > 0 and isinstance(prompts[0], str):
                 # If prompts are strings, use the first one
                 question_text = prompts[0]


            print(
                '*'*20 + f"Question:\n{question_text}",
                f"\nAnswer:\n{true_answers[0]}", # Access first true answer
                f"\nResponse:\n{responses[0]}", # Access first generated response
                f"\nExtracted:\n{extracted_responses[0] if extracted_responses else 'None'}" # Access first extracted response
            )
        except Exception as e:
            print(f"Error printing debug info: {e}")
            print(f"Prompts structure: {prompts}")
            print(f"Responses structure: {responses}")
            print(f"Extracted responses structure: {extracted_responses}")
            print(f"True answers structure: {true_answers}")


    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, true_answers):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            # Ensure true_answer is a string before stripping and replacing
            true_answer_str = str(true_answer).strip()
            guess_str = str(guess).strip().replace(",", "")

            true_answer_float = float(true_answer_str)
            guess_float       = float(guess_str)

            scores.append(3.5 if guess_float == true_answer_float else -1.5)
        except:
            # If conversion to float fails, assign a neutral or penalty score
            scores.append(0) # Or a small penalty like -0.5
            continue
    return scores

Get the top 90% prompt length so we don't accidentally truncate them!

Ie we'll remove the top 10% long prompts.

In [ ]:
tokenized_train = train_dataset_mcq.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["Messages"], add_generation_prompt = True, tokenize = True)},
    batched = False, # Set batched to False to process row by row if batched=True causes issues
)
print(tokenizer.decode(tokenized_train[0]["tokens"]))
tokenized_train = tokenized_train.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized_train["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length for training set
train_dataset_mcq = train_dataset_mcq.select(np.where(np.array(tokenized_train["L"]) <= maximum_length)[0])
del tokenized_train

# Tokenize and filter the dev dataset separately
tokenized_dev = dev_dataset_mcq.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["Messages"], add_generation_prompt = True, tokenize = True)},
    batched = False,
)
tokenized_dev = tokenized_dev.map(lambda x: {"L" : len(x["tokens"])})

# Determine maximum length for dev set based on its own data (or reuse maximum_length from train if preferred)
# Here, we'll reuse maximum_length from the training set for consistency
dev_dataset_mcq = dev_dataset_mcq.select(np.where(np.array(tokenized_dev["L"]) <= maximum_length)[0])
del tokenized_dev

Map:   0%|          | 0/18944 [00:00<?, ? examples/s]

أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين <start_reasoning> و <end_reasoning>.
ثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>مات وترك: أخت لأم و أخ شقيق (2) و ابن عم الأب (3) و ابن ابن عم لأب (4) و أم الأب و أم أب الأب كم النصيب الأصلي لـ أخت لأم من التركة، وما الدليل على ذلك؟<start_reasoning>نصيبه هو السدس، والدليل: فرض الواحد - المنفرد - من الأخوة لأم هو السدس سواء أكان ذكرا أو أنثى ، وشرط ميراثهم هو عدم وجود الفرع الوارث المذكر - مثل الإبن وابن الإبن - أو المؤنث - مثل البنت وبنت الإبن - وعدم وجود الأصل الوارث المذكر - مثل الأب و أب الأب<end_reasoning><SOLUTION>A</SOLUTION><|im_end|><start_reasoning>


Map:   0%|          | 0/18944 [00:00<?, ? examples/s]

Max Length =  209


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 100,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 500,
    # save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "/content/drive/MyDrive/PalmX/GRPO",
    save_total_limit=2,

    # # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 1,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 2 to the `num_generations` of 4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = train_dataset_mcq,
    eval_dataset = dev_dataset_mcq
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17,078 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 1,056,964,608 of 5,079,432,704 (20.81% trained)
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:660: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 2560], which does not match the required output shape [4, 1, 2560]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Trig

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
100,0.000200,4.870000,2.047944,310.505000,169.290000,548.180000,0.012500,265.373334,169.290000,372.320000,0.195594,2.707500,0.380885,1.252500,0.348956,3.178750,1.526700,-2.268750,0.191368
200,0.000300,6.491250,0.676741,194.382500,111.760000,317.590000,0.015000,138.540000,111.760000,170.320000,0.349340,2.917500,0.126962,1.406250,0.143122,4.617500,0.474959,-2.450000,0.039434


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


********************Question:
أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين <start_reasoning> و <end_reasoning>.
ثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>مات وترك: أخ لأب (5) و أب الأب و ابن عم لأب (5) كم النصيب الأصلي لكل صنف من الورثة من التركة؟<start_reasoning>أب الأب: الثلث، أخ لأب (5): باقى التركة، ابن عم لأب(5): محجوب<end_reasoning><SOLUTION>E</SOLUTION><|im_end|> 
Answer:
E 
Response:


</think>

<start_reasoning>
أولاً، نحدد الورثة بناءً على القوانين الشرعية:

1. **أب الأب**: من أب الورثة، له حق النصيب من التركة.
2. **أخ لأب (5)**: أخ من أب، له حق في التركة.
3. **ابن عم لأب (5)**: ابن عم من أب، وهو محجوب (لا يحق له النصيب من التركة).

النسبة الموصى بها للورثة:
- **أب الأب** يحصل على **الثلث** من التركة.
- **أخ لأب (5)** يحصل على **الباقي** بعد توزيع نصيب أب الأب.
- **ابن عم لأب (5)**: محجوب، لا يحصل على نصيب.

إذن، النصيب الأصلي:
- **أب الأب**: الثلث.
- **أخ لأب (5)**: الباقي من التركة (أي مجموع التركة ناقص الثلث).
- **ابن عم لأب (5)**: لا يحصل عل

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

********************Question:
أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين <start_reasoning> و <end_reasoning>.
ثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>توفيت عن: (بنت) العدد 2، ابنة أخيهم المتوفى الوحيدة.
ما هو مجموع نصيب البنتين من التركة مقدرًا بالكسر؟<start_reasoning>F) الثلثان<end_reasoning><SOLUTION>F</SOLUTION><|im_end|> 
Answer:
F 
Response:


</think>

<start_reasoning>
في هذه المسألة، نحتاج إلى تحديد نصيب البنتين من التركة مقدرًا بالكسر. وفقًا للقواعد الشرعية، تُعتبر البنتان من أبناء الأخ الأكبر. لذلك، نصيب البنتين من التركة هو الثلثان.<end_reasoning>

<SOLUTION>F</SOLUTION> 
Extracted:
None


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


********************Question:
أنت أمام مسألة معينة.
فكّر في المسألة وقدم شرح خطوات تفكيرك.
ضعها بين <start_reasoning> و <end_reasoning>.
ثم، قدم الحل بين <SOLUTION></SOLUTION><|im_end|>مات وترك: ابن ابن أخ لأب (5) و أم الأم و أخ شقيق و ابن ابن أخ شقيق و بنت ابن و أب الأب، كم عدد الأسهم بعد التصحيح التي تحصل عليها أم الأم؟<start_reasoning>1 سهم<end_reasoning><SOLUTION>D</SOLUTION><|im_end|> 
Answer:
D 
Response:


</think>

<start_reasoning>
في هذه المسألة، نحتاج إلى تحديد عدد الأسهم بعد التصحيح التي تحصل عليها أم الأم. وفقًا للنص، نجد أن أم الأم تمتلك 1 سهم. بعد التصحيح، عدد الأسهم التي تحصل عليها أم الأم هو 1 سهم.<end_reasoning><SOLUTION>D</SOLUTION> 
Extracted:
None


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/unsloth/Qwen3-4B/resolve/main/config.json (Request ID: Root=1-68845e01-116b713874482d8b5d192b85;84707a0c-6fc8-410f-88fc-57c9af2ff716)

Invalid credentials in Authorization header - silently ignoring the lookup for the file config.json in unsloth/Qwen3-4B.
  warnings.warn(
/usr/local/

TrainOutput(global_step=500, training_loss=0.00032135539874434473, metrics={'train_runtime': 11453.3659, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.044, 'total_flos': 0.0, 'train_loss': 0.00032135539874434473})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "مات وترك: أخت لأب (2) و أخت شقيقة (4) و ابن عم شقيق (4) و أم، كم عدد الأسهم التي يحصل عليها كل صنف من الورثة قبل تصحيح المسألة؟"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)

ValueError: The following `model_kwargs` are not used by the model: ['sampling_params'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
output[0].outputs[0].text

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "مات وترك: أخت لأب (2) و أخت شقيقة (4) و ابن عم شقيق (4) و أم، كم عدد الأسهم التي يحصل عليها كل صنف من الورثة قبل تصحيح المسألة؟"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
token = ""

# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = token)
    tokenizer.push_to_hub("hf/model", token = token)


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = '/content/drive/MyDrive/PalmX/GRPO/checkpoint-500',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token =''
)

/tmp/ipython-input-6-2534213347.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-26 09:34:41 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-26 09:34:41 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.53.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import re
import zipfile
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
letter_choices = ["A", "B", "C", "D", "E", "F"]
max_new_tokens = 1024
team_name = "Gumball"

submission_data = []

for idx, example in tqdm(enumerate(test_dataset_mcq), total=len(test_dataset_mcq)):
    # Format the example into the expected message list structure
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": example["question"]},
    ]
    chat_prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer(chat_prompt, return_tensors="pt", truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id
        )

    new_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    predicted_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print(idx)
    print(predicted_text)

    match = re.search(r"\b([A-F])\b", predicted_text.upper())
    prediction = match.group(1) if match else "F"
    # حفظ النتيجة
    submission_data.append({
        "id_question": example["id_question"],
        "prediction": prediction
    })

# إنشاء ملف CSV
df_submission = pd.DataFrame(submission_data)
csv_filename = f"{root_path}/prediction.csv"
df_submission.to_csv(csv_filename, index=False, encoding="utf-8")

# ضغط الملف
zip_filename = f"{root_path}/subtask1_{team_name}_predictions.zip"
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

print(f"\n✅ Submission file created: {zip_filename}")

  0%|          | 0/1000 [00:00<?, ?it/s]

0
في هذه المسألة، نحتاج إلى تحديد عدد الأسهم التي تحصل عليها بنت (5) قبل تصحيح المسألة. نبدأ بتحليل العلاقات الأسرية. ابن (5) هو ابن ابن عم، مما يعني أن ابن عم هو شقيق الأب. بنت (5) هي ابنة ابن عم، وبالتالي، هي شقيقة لابن عم. الآن، نحتاج إلى تحديد عدد الأسهم التي تحصل عليها بنت (5) قبل تصحيح المسألة. في الأسرة، هناك عدد معين من الأشخاص، ونحتاج إلى تحديد عدد الأسهم التي تحصل عليها بنت (5) بناءً على العلاقات الأسرية. بعد التفكير في العلاقات الأسرية، نجد أن بنت (5) تحصل على 3 أوراق. <end_reasoning> <SOLUTION> 3 </SOLUTION>
1
Okay, let's try to figure out this problem. So, the question is about determining the total number of shares (sahām) that the inheritance is divided among before correcting the issue. The problem mentions two people: the son-in-law of the father (5) and the mother of the father and the son of the father's brother (4). 

First, I need to understand the relationships here. The first person is the son-in-law of the father, which means he is married to the daughter of the

KeyboardInterrupt: 

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model_2, tokenizer_2 = FastLanguageModel.from_pretrained(
    model_name = '/content/drive/MyDrive/PalmX/GRPO/checkpoint-500',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token =''
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-26 09:49:15 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-26 09:49:15 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.53.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.7.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
HF_TOKEN = ""
model_name = "unsloth/Qwen3-0.6B"
letter_choices = ["A", "B", "C", "D","E","F"]
max_new_tokens =   # enough for a full sentence answer
FastLanguageModel.for_inference(model2)
# model2 = model2.to(device)
dataset = load_dataset("csv", data_files={"dev": "/content/Qias/Task1_MCQs_Dev.csv}, split="dev")
subset = dataset.select(range(100))
def format_chat_prompt(example):
    prompt = f''' "ما هي الإجابة الصحيحة على السؤال التالي؟ .\n\n"
        "السؤال: {example['question']}\n"
        الاختيارات :
        "A. {example['option1']}\n"
        "B. {example['option2']}\n"
        "C. {example['option3']}\n"
        "D. {example['option4']}\n"
        "E. {example['option5']}\n"
        "F. {example['option6']}\n"
    جاوب برمز الاجابه الصحيحه فقط
    '''.replace('\n' , '')
    messages =[]
    messages.append({'role' : 'system' , 'content': "أنت مساعد ذكي تتحدث باللغة العربية الفصحى. كن مهنيًا، لبقا وودودا في ردودك، وتحدث دائما بالفصحى. أجب بوضوح وتفصيل، وتجنب الردود المختصرة."})
    messages.append({"role": "user", "content": prompt})
    print(messages)
    chat_text = tokenizer2.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,
    )
    return chat_text
correct = 0
total = len(subset)
for idx, example in enumerate(subset):
    chat_prompt = format_chat_prompt(example)
    inputs = tokenizer2(chat_prompt, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        outputs = model2.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id
        )
    new_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    predicted_text = tokenizer2.decode(new_tokens, skip_special_tokens=True).strip()
    match = re.search(r"\b([A-F])\b", predicted_text.upper())
    prediction = match.group(1) if match else "?"
    is_correct = prediction == example["label"]
    status = ":white_check_mark:" if is_correct else ":x:"
    print(f"\n[{idx+1}/{total}]")
    print(f"السؤال: {example['question']}")
    print(f"Correct answer: {example['label']}")
    print(f"Raw generated answer: {predicted_text}")
    print(f"Predicted letter (if any): {prediction} → {status}")
    if is_correct:
        correct += 1
accuracy = correct / total
print(f"\n:white_check_mark: Final Accuracy ({total} samples): {accuracy:.2%}")

SyntaxError: unterminated string literal (detected at line 12) (ipython-input-21-2024322575.py, line 12)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
import re
import zipfile

device = "cuda" if torch.cuda.is_available() else "cpu"
letter_choices = ["A", "B", "C", "D", "E", "F"]
max_new_tokens = 50
team_name = "Gumball"

dataset = load_dataset("csv", data_files={"dev": "/content/Qias/Task1_MCQs_Test.csv"}, split="dev")
subset = dataset

submission_data = []

def format_chat_prompt(example):
    prompt = f'''
    ما هي الإجابة الصحيحة على السؤال التالي؟
    السؤال: {example['question']}
    الاختيارات:
    A. {example['option1']}
    B. {example['option2']}
    C. {example['option3']}
    D. {example['option4']}
    E. {example['option5']}
    F. {example['option6']}
    جاوب برمز الإجابة الصحيحة فقط.
    '''.replace('\n', ' ')

    messages = [
        {'role': 'system', 'content': "أنت مساعد ذكي تتحدث باللغة العربية الفصحى. كن مهنيًا، لبقا وودودا في ردودك."},
        {'role': 'user', 'content': prompt}
    ]

    chat_text = tokenizer_2.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,
    )
    return chat_text

for idx, example in enumerate(subset):
    chat_prompt = format_chat_prompt(example)
    inputs = tokenizer_2(chat_prompt, return_tensors="pt", truncation=True).to(device)

    with torch.no_grad():
        outputs = model_2.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer_2.eos_token_id
        )

    new_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    predicted_text = tokenizer_2.decode(new_tokens, skip_special_tokens=True).strip()
    print(idx)
    # print(predicted_text)

    match = re.search(r"\b([A-F])\b", predicted_text.upper())
    prediction = match.group(1) if match else "F"
    print(prediction)
    # حفظ النتيجة
    submission_data.append({
        "id_question": example["id_question"],
        "prediction": prediction
    })

# إنشاء ملف CSV
df_submission = pd.DataFrame(submission_data)
csv_filename = f"prediction.csv"
df_submission.to_csv(csv_filename, index=False, encoding="utf-8")

# ضغط الملف
zip_filename = f"subtask1_{team_name}_predictions.zip"
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

print(f"\n✅ Submission file created: {zip_filename}")


0
A
1
F
2
A
3
D
4
B
5
B
6
A
7
A
8
F
9
A
10
A
11
C
12
A
13
A
14
A
15
D
16
A
17
C
18
E
19
F
20
D
21
A
22
F
23
B
24
A
25
B
26
F
27
D
28
A
29
D
30
A
31
A
32
A
33
D
34
D
35
A
36
A
37
D
38
C
39
E
40
A
41
A
42
F
43
B
44
A
45
A
46
A
47
E
48
E
49
A
50
A
51
F
52
D
53
A
54
B
55
A
56
A
57
A
58
D
59
A
60
A
61
B
62
A
63
C
64
D
65
A
66
E
67
F
68
F
69
A
70
A
71
F
72
C
73
A
74
D
75
B
76
D
77
E
78
F
79
A
80
D
81
A
82
F
83
E
84
C
85
C
86
A
87
B
88
A
89
B
90
E
91
F
92
B
93
E
94
A
95
D
96
E
97
F
98
B
99
D
100
A
101
F
102
A
103
A
104
D
105
E
106
A
107
F
108
F
109
A
110
F
111
D
112
D
113
D
114
D
115
A
116
A
117
F
118
E
119
C
120
A
121
E
122
A
123
D
124
E
125
D
126
D
127
E
128
F
129
D
130
A
131
D
132
A
133
A
134
B
135
A
136
C
137
F
138
A
139
B
140
B
141
D
142
A
143
E
144
A
145
E
146
A
147
F
148
E
149
F
150
A
151
A
152
A
153
B
154
E
155
F
156
A
157
A
158
A
159
E
160
B
161
A
162
D
163
A
164
A
165
F
166
A
167
C
168
A
169
A
170
A
171
E
172
A
173
A
174
F
175
D
176
A
177
A
178
B
179
A
180
A
181
C
182
D
183
A
184
A


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
HF_TOKEN = ""
letter_choices = ["A", "B", "C", "D","E","F"]
max_new_tokens = 50  # enough for a full sentence answer
FastLanguageModel.for_inference(model_2)
# model2 = model2.to(device)
dataset = load_dataset("csv", data_files={"dev": "/content/Qias/Task1_MCQs_Dev.csv"}, split="dev")
subset = dataset.select(range(1000))
def format_chat_prompt(example):
    prompt = f''' "ما هي الإجابة الصحيحة على السؤال التالي؟ .\n\n"
        "السؤال: {example['question']}\n"
        الاختيارات :
        "A. {example['option1']}\n"
        "B. {example['option2']}\n"
        "C. {example['option3']}\n"
        "D. {example['option4']}\n"
        "E. {example['option5']}\n"
        "F. {example['option6']}\n"
    جاوب برمز الاجابه الصحيحه فقط
    '''.replace('\n' , '')
    messages =[]
    messages.append({'role' : 'system' , 'content': "أنت مساعد ذكي تتحدث باللغة العربية الفصحى. كن مهنيًا، لبقا وودودا في ردودك."})
    messages.append({"role": "user", "content": prompt})
    print(messages)
    chat_text = tokenizer_2.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True,
    )
    return chat_text
correct = 0
total = len(subset)
for idx, example in enumerate(subset):
    chat_prompt = format_chat_prompt(example)
    inputs = tokenizer_2(chat_prompt, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        outputs = model_2.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer_2.eos_token_id
        )
    new_tokens = outputs[0][inputs['input_ids'].shape[-1]:]
    predicted_text = tokenizer_2.decode(new_tokens, skip_special_tokens=True).strip()
    match = re.search(r"\b([A-F])\b", predicted_text.upper())
    prediction = match.group(1) if match else "?"
    is_correct = prediction == example["label"]
    status = ":white_check_mark:" if is_correct else ":x:"
    print(f"\n[{idx+1}/{total}]")
    print(f"السؤال: {example['question']}")
    print(f"Correct answer: {example['label']}")
    print(f"Raw generated answer: {predicted_text}")
    print(f"Predicted letter (if any): {prediction} → {status}")
    if is_correct:
        correct += 1
accuracy = correct / total
print(f"\n:white_check_mark: Final Accuracy ({total} samples): {accuracy:.2%}")

Streaming output truncated to the last 5000 lines.
Raw generated answer: </start_reasoning>
Answer: B

The question is about determining the share of the wife in an inheritance scenario. Let's break down the information given:

- The person has 4 sons.
- 2 brothers (siblings).
-
Predicted letter (if any): B → :white_check_mark:
[{'role': 'system', 'content': 'أنت مساعد ذكي تتحدث باللغة العربية الفصحى. كن مهنيًا، لبقا وودودا في ردودك.'}, {'role': 'user', 'content': ' "ما هي الإجابة الصحيحة على السؤال التالي؟ ."        "السؤال: توفي عن زوجة، وسبعة أولاد، وبنتان، وعشرة أبناء ابن، واثنتاعشرة بنت ابن، وأربعة أبناء ابن ابن، وأخ شقيق، وأحد عشر ابن أخ شقيق. ما نصيب الزوجة؟"        الاختيارات :        "A. A) الربع"        "B. B) الثمن"        "C. C) السدس"        "D. D) النصف"        "E. E) الثلث"        "F. F) لا ترث"    جاوب برمز الاجابه الصحيحه فقط    '}]

[552/1000]
السؤال: توفي عن زوجة، وسبعة أولاد، وبنتان، وعشرة أبناء ابن، واثنتاعشرة بنت ابن، وأربعة أبناء ابن ابن، وأخ شقيق، وأحد عشر ابن أ